In [1]:
import os
# import sys

# from src.processing import mean_pooling, mean_pooling_embedding_with_normalization

# sys.path.append('.')

import argparse
import json
import numpy as np

import faiss
import torch
# from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

/Users/amarkanaka/miniconda3/envs/faiss_1.8.0_new_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
model = AutoModel.from_pretrained('facebook/contriever')

In [3]:
tokenizer

BertTokenizerFast(name_or_path='facebook/contriever', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [4]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [7]:
# Embeddings Model is a mapping from chunks of max length 512 tokens -> R^768 ???
dim = 768
#normalize embeddings
norm = True
dataset = 'musique'
# unit = 'proposition'

In [4]:
model_label = 'facebook_contriever'

In [8]:
vector_path = f'data/{dataset}/{dataset}_{model_label}_proposition_vectors_norm.npy'
index_path = f'data/{dataset}/{dataset}_{model_label}_proposition_ip_norm.index'
if(os.path.isfile(vector_path)):
    vectors = np.load(vector_path)


In [9]:
vector_path

'data/musique/musique_facebook_contriever_proposition_vectors_norm.npy'

In [14]:
if dataset == 'musique':
    corpus = json.load(open('data/musique_proposition_corpus.json', 'r'))
elif dataset == '2wikimultihopqa':
    corpus = json.load(open('data/2wikimultihopqa_proposition_corpus.json', 'r'))
corpus_contents = []
for item in corpus:
    corpus_contents.append(item['title'] + '\n' + item['propositions'])
print('corpus size: {}'.format(len(corpus_contents)))

corpus size: 11656


In [18]:
total_len = 0
max_len = 0
min_len = 1000000
for line in corpus_contents:
    total_len += len(line)
    if len(line) > max_len:
        max_len = len(line)
    if len(line) < min_len:
        min_len = len(line)
print(max_len / 4)
print(min_len / 4)
print((total_len / len(corpus_contents)) / 4)    

525.25
31.25
149.56335792724778


In [19]:
if os.path.isfile(vector_path):
    print('Loading existing vectors:', vector_path)
    vectors = np.load(vector_path)
    print('Vectors loaded:', len(vectors))